In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf()
sc = SparkContext(conf=conf)
from collections import Counter
import itertools

#Name of the file for the dataset
fileName = 'soc-LiveJournal1Adj.txt'
#set N=10 inorder to get the list of only 10 suggestions for friends
N = 10

def friends_mutual_connected(line):
    #setting minimum so that we don't consider users who are already friends
    minimum = -9999999999
    #getting values for user and their friends
    user, friends = line.split('\t')
    friends = friends.split(',')
    #getting users who are already friends
    connected_friends = [((user, friend), minimum) for friend in friends]
    #getting mutual friends
    mutual_friends = [(pair, 1) for pair in itertools.permutations(friends, 2)]
    return connected_friends + mutual_friends

def filter_to_reject(a):
    if a[1]>0:
        return True
    else:
        return False


def mapping(a):
    answer=(a[0][0],(a[1],a[0][1]))
    return answer

#creating a function to find suggestions of friends for users
def recommendations(a):
    user=a[0]
    suggestions=a[1]
    answer=(user, Counter( dict( (friend, count) for count, friend in suggestions ) ).most_common( N ) )
    return answer


suggestionsRDD = (sc
                  .textFile(fileName, 16)
                  .flatMap(friends_mutual_connected)
                  .reduceByKey(lambda total, current: total + current)  
                  .filter(filter_to_reject)  #filter to reject users who are already friends
                  .map(mapping)
                  .groupByKey()
                  .map(recommendations)
               )
			   
print (suggestionsRDD.lookup('8941'))

In [ ]:
#creating a list for users of which recommendations are asked
list=['924', '8941', '8942', '9019', '9020', '9021', '9022', '9990', '9992', '9993']

In [ ]:
#printing the answer
for i in range(len(list)):
    print(list[i])
    print (suggestionsRDD.lookup(list[i]))
    print()

In [ ]:
sc.stop()